In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime

2025-07-03 19:03:42.618651: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 19:03:42.770755: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-03 19:03:42.770986: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-03 19:03:42.795452: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 19:03:42.843449: I tensorflow/core/platform/cpu_feature_guar

In [3]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ActivityRegularization, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from module import Ranger

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
# gpu settings

gpus = tf.config.list_physical_devices("GPU")

if gpus:
    print("GPU detected: ", gpus[0])
    try:
        tf.config.set_visible_devices(gpus[0], "GPU")
        ldev = tf.config.list_logical_devices("GPU")
        print(len(ldev), "Logical devices and ", len(gpus), "Physical devices detected")
    except RuntimeError as e:
        print(e)

if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except:
        pass

os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
print(os.getenv("TF_GPU_ALLOCATOR"))

tf.debugging.set_log_device_placement(True)

GPU detected:  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
1 Logical devices and  1 Physical devices detected
cuda_malloc_async


2025-07-03 19:03:45.986589: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-03 19:03:46.093579: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-03 19:03:46.093742: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
train_dir = "/home/abhinav/Documents/2 Work/2 Hobby_projects/Datasets/Imagenet/train_synsets"
valid_dir = "/home/abhinav/Documents/2 Work/2 Hobby_projects/Datasets/Imagenet/valid_synsets"
test_dir = "/home/abhinav/Documents/2 Work/2 Hobby_projects/Datasets/Imagenet/test_synsets"

model_dir = "/home/abhinav/Documents/2 Work/2 Hobby_projects/Models"

In [6]:
num_epochs = 1
batch_size = 20
img_h = 128
img_w = 128
num_channels = 3
num_classes = 3

In [7]:
# class MyNet(Sequential):
#     def __init__(self, input_shape, num_classes):
#         super().__init__()

#         self.add(Conv2D(
#             70,
#             kernel_size=(3, 3),
#             strides=1,
#             padding="valid",
#             kernel_initializer="he_normal",
#             input_shape=input_shape,
#             activation="relu"
#         ))

#         self.add(Conv2D(
#             128,
#             kernel_size=(3, 3),
#             strides=2,
#             padding="valid",
#             kernel_initializer="he_normal",
#             activation="relu"
#         ))
        
#         self.add(MaxPooling2D(
#             pool_size=(3, 3),
#             strides=2,
#             padding="valid",
#             data_format=None
#         ))

#         self.add(Conv2D(
#             256,
#             kernel_size=(3, 3),
#             strides=1,
#             padding="valid",
#             kernel_initializer="he_normal",
#             activation="relu"
#         ))

#         self.add(Conv2D(
#             256,
#             kernel_size=(3, 3),
#             strides=1,
#             padding="valid",
#             kernel_initializer="he_normal",
#             activation="relu"
#         ))

#         self.add(MaxPooling2D(
#             pool_size=(3, 3),
#             strides=2,
#             padding="valid",
#             data_format=None
#         ))

#         self.add(Conv2D(
#             256,
#             kernel_size=(3, 3),
#             strides=1,
#             padding="valid",
#             kernel_initializer="he_normal",
#             activation="relu"
#         ))

#         self.add(Conv2D(
#             512,
#             kernel_size=(5, 5),
#             strides=2,
#             padding="valid",
#             kernel_initializer="he_normal",
#             activation="relu"
#         ))

#         self.add(Conv2D(
#             64,
#             kernel_size=(5, 5),
#             strides=2,
#             padding="valid",
#             kernel_initializer="he_normal",
#             activation="relu"
#         ))

#         # self.add(BatchNormalization(momentum=1))
#         self.add(Flatten())
#         # self.add(Dropout(0.5))
#         # self.add(Dense(4096, activation="relu"))
#         self.add(Dropout(0.4))
#         # self.add(Dense(3796, activation="relu"))
#         self.add(Dense(1000, activation="relu"))
#         self.add(Dense(num_classes, activation="softmax"))

#         self.compile(
#             # optimizer=Ranger(learning_rate=0.0001),
#             # optimizer=Ranger(learning_rate=0.000001, amsgrad=True, slow_step_size=0.8, warmup_proportion=0.3),
#             optimizer=Adam(learning_rate=0.0001, use_ema=True, ema_momentum=1),
#             loss="categorical_crossentropy",
#             metrics=["accuracy"]
#         )


In [8]:
class MyNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        self.add(Conv2D(
            256,
            kernel_size=(3, 3),
            strides=1,
            padding="valid",
            kernel_initializer="he_normal",
            input_shape=input_shape,
            activation="relu"
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3, 3),
            strides=1,
            padding="valid",
            kernel_initializer="he_normal",
            activation="relu"
        ))
        
        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides=2,
            padding="valid",
            data_format=None
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3, 3),
            strides=1,
            padding="valid",
            kernel_initializer="he_normal",
            activation="relu"
        ))


        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides=2,
            padding="valid",
            data_format=None
        ))


        self.add(Conv2D(
            32,
            kernel_size=(3, 3),
            strides=2,
            padding="valid",
            kernel_initializer="he_normal",
            activation="relu"
        ))

        # self.add(BatchNormalization(momentum=1))
        self.add(Flatten())
        # self.add(Dropout(0.5))
        # self.add(Dense(4096, activation="relu"))
        self.add(Dropout(0.4))
        # self.add(Dense(3796, activation="relu"))
        self.add(Dense(1000, activation="relu"))
        self.add(Dense(num_classes, activation="softmax"))

        self.compile(
            # optimizer=Ranger(learning_rate=0.0001),
            # optimizer=Ranger(learning_rate=0.000001, amsgrad=True, slow_step_size=0.8, warmup_proportion=0.3),
            optimizer=Adam(learning_rate=0.0001, use_ema=True, ema_momentum=1),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )


In [9]:
model = MyNet((img_w, img_h, num_channels), num_classes)
model.summary()

Model: "my_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 256)     7168      
                                                                 
 conv2d_1 (Conv2D)           (None, 124, 124, 128)     295040    
                                                                 
 max_pooling2d (MaxPooling2  (None, 61, 61, 128)       0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 59, 59, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 29, 29, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 32)        36896

In [10]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    # rotation_range=2,
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    # shear_range=0.6,
    # zoom_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_w, img_h),
    color_mode="rgb",
    batch_size=batch_size,
    seed=1,
    shuffle=True,
    class_mode="categorical"
)

valid_datagen = ImageDataGenerator(
    rescale=1./255
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_w, img_h),
    color_mode="rgb",
    batch_size=batch_size,
    seed=7,
    shuffle=True,
    class_mode="categorical"
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_w, img_h),
    color_mode="rgb",
    batch_size=batch_size,
    seed=7,
    shuffle=True,
    class_mode="categorical"
)

train_num = train_generator.samples
valid_num = valid_generator.samples
train_num = train_generator.samples

Found 2495 images belonging to 3 classes.
Found 1071 images belonging to 3 classes.
Found 714 images belonging to 3 classes.


In [22]:
logs_dir = "./logs/fit" + datetime.datetime.now().strftime("%d%M%Y-%H%M%S")
tensorboard_callbacks = tf.keras.callbacks.TensorBoard(log_dir=logs_dir)
callbacks_list = [tensorboard_callbacks]

model.fit(
    train_generator,
    epochs=num_epochs,
    batch_size=batch_size,
    validation_data=valid_generator,
    callbacks=callbacks_list,
    verbose=1
)

125/125 [==============================] - 29s 233ms/step - loss: 0.3984 - accuracy: 0.8413 - val_loss: 0.3625 - val_accuracy: 0.8543


In [21]:
results = model.evaluate(
    test_generator,
    verbose=1
)

print(f"Accuracy obtained: {results[1]*100:.3f}")

36/36 [==============================] - 2s 67ms/step - loss: 0.4708 - accuracy: 0.8053
Accuracy obtained: 80.532


In [17]:
# model.save(os.path.join(model_dir, "Mynet1"))